In [ ]:
!pip install sentencepiece
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.7 MB/s eta 0:00:00


In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz

--2023-05-03 05:08:10--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/train-clean-100.tar.gz [following]
--2023-05-03 05:08:10--  http://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  19.3MB/s    in 5m 26s  

2023-05-03 05:13:37 (18.7 MB/s) - ‘train-clean-100.tar.gz’ saved [6387309499/6387309499]



In [ ]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz

--2023-05-13 10:45:28--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/test-clean.tar.gz [following]
--2023-05-13 10:45:29--  http://us.openslr.org/resources/12/test-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  28.8MB/s    in 13s     

2023-05-13 10:45:42 (26.2 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

--2023-05-13 10:45:42--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/dev-clean.tar.gz [following]
--2023-05-13 10:45:42--  http://us.openslr.org/resources/12/dev-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  28.9MB/s    in 12s     

2023-05-13 10:45:55 (26.2 MB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]



In [ ]:
!tar xvzf train-clean-100.tar.gz

In [ ]:
!tar xvzf dev-clean.tar.gz

In [ ]:
!tar xvzf test-clean.tar.gz

In [ ]:
import torch
from torch import nn
import numpy as np
from torchaudio.models import RNNT,Emformer,emformer_rnnt_base,emformer_rnnt_model
from torch import optim
import torch
from torch import nn
from torchaudio import functional as audioF
from torchaudio import transforms as audioT
import torchaudio
import Levenshtein as Lev

In [ ]:
import numpy as np
import torch
from torch import nn
from glob import glob
import librosa
from tqdm import tqdm
import soundfile as sf
import os
import random
from pathlib import Path

In [ ]:
def wer(s1,s2):
  b = set(s1.split()+s2.split())
  word_map = dict(zip(b,range(len(b))))
  print("word_map:",word_map)
  w1 = [chr(word_map[w]) for w in s1.split()]
  w2 = [chr(word_map[w]) for w in s2.split()]
  dev = max(len(s2.split()),1)
  score = Lev.distance("".join(w1),"".join(w2))/dev
  return score

def cer(s1,s2):
  w1 = s1.replace(" ","")
  w2 = s2.replace(" ","")
  dev = max(len(s2),1)
  score = Lev.distance((w1),(w2))/dev
  return score

In [ ]:
class DataLoader:
    def __init__(self,batch_size=1,is_test_set=False):
        super().__init__()
        self.batch_size = batch_size
        self.is_test_set = is_test_set
        common_folder = "LibriSpeech/dev-clean"#"LibriSpeech/train-clean-100"
        if(is_test_set==True):
          common_folder = "LibriSpeech/test-clean"
        transcripts_files = glob(os.path.join(common_folder,"*/*/*.txt"),recursive=True)
        audio_files = glob(os.path.join(common_folder,"*/*/*.flac"),recursive=True)
        if(is_test_set==False):
          audio_files_len = len(audio_files)
          audio_files = audio_files[:audio_files_len//25]
        print("transcripts_files:",len(transcripts_files))
        print("audio_files:",len(audio_files))
        transcripts = {}
        for ts_fpath in transcripts_files:
          with open(ts_fpath,"r") as fp:
              txt = fp.read().split("\n")
              for ln in txt:
                  ln=ln.strip()
                  if(ln==""):
                      continue
                  
                  lines = ln.split(" ",1)
                  fn = lines[0]
                  lbls = lines[1].upper()
                  transcripts[fn] = lbls
        self.audio_files = audio_files
        self.transcript_keys = list(transcripts.keys())
        self.transcript_table = "".join([chr(ord("A")+i) for i in range(26)])+(" .'!?")
        #print("transcript",transcripts)
        #print("transcript keys:",self.transcript_keys)
        
        self.transcripts = transcripts
        self._audio_file_len = len(self.audio_files)
        if(is_test_set==False):
          self.shuffleDataset()

    def __len__(self):
        return len(self.audio_files)//self.batch_size
    def _loadFeature(self,fpath):
        rawinput,sample_rate = sf.read(fpath)
        rawinput = abs(rawinput)
        mfcc = librosa.feature.mfcc(y=rawinput,sr=sample_rate,n_mfcc=96).T
        mfcc = np.pad(mfcc,((0,4),(0,0)))# emformer rnn-t 가 이전 문맥과 자연스럽게 연결짓기 위해서 right context padding 을 가짐
        return mfcc
    def _convertTranscript2(self,text_list,with_blank=False):
        #transcript_table: "".join([chr(ord("A")+i) for i in range(26)])+(" .'!?")
        #               => [A~Z] & " .'!?"
        result = []
        for char in text_list:
            result.append(self.transcript_table.find(char)+3)
            if(with_blank==True):
              result.append(2)
        return result
    def shuffleDataset(self):
        random.shuffle(self.audio_files)
    def token2String(self,token_list,with_visible_tag=True):
      result = ""
      max_len = len(self.transcript_table)+2
      for i in range(len(token_list)):
        token_id = token_list[i]
        if(token_id==1):
          result+='<s>' if(with_visible_tag==True) else ""
        elif(token_id==0):
          result += '</s>' if(with_visible_tag==True) else ""
        elif(token_id==2):
          result += '</b>' if(with_visible_tag==True) else ""
        elif(2<token_id<max_len):
          result+= self.transcript_table[token_id-3]
        else:
          result+='</u?>' if(with_visible_tag==True) else ""
      return result
    
    def __getitem__(self,idx):
        if(idx>=self.__len__()):
            raise StopIteration()
        start_index,end_cond = (idx*self.batch_size,min((idx+1)*self.batch_size,(self._audio_file_len)))
        #print((start_index,end_cond))
        specs = []
        transcripts = []
        transcript_lengths = []
        specs_lengths = []

        for i in range(start_index,end_cond):
            fpath = self.audio_files[i]
            feature = self._loadFeature(fpath)
            fn = Path(fpath).stem
            specs.append(torch.from_numpy(feature))
            specs_lengths.append(feature.shape[0])
            transcript_indices = [0]+self._convertTranscript2(self.transcripts[fn],with_blank=False)
            transcripts.append(torch.from_numpy(np.array(transcript_indices)))
            transcript_lengths.append(len(transcript_indices)))
        transcript_lengths = np.array(transcript_lengths)
        specs_lengths = np.array(specs_lengths)
        return specs,specs_lengths,transcripts,transcript_lengths

In [ ]:
if(__name__=="__main__"):
    traindataset_loader = DataLoader(15)
    from matplotlib import pyplot as plt
    index = 0
    for specs,specs_lengths,transcripts,transcript_lengths in tqdm(traindataset_loader):
        print(specs.shape,specs_lengths.shape,transcripts.shape,transcript_lengths.shape)
        fig, ax = plt.subplots(nrows=2, sharex=True)
        mfcc= specs[0].T
        print("\t",(np.min(mfcc),np.max(mfcc)))
        img = librosa.display.specshow(librosa.power_to_db(mfcc, ref=np.max),
                                      x_axis='time', y_axis='mel',fmin=100, fmax=8000,
                                      ax=ax[0])
        fig.colorbar(img, ax=[ax[0]])
        ax[0].set(title='Mel spectrogram')
        ax[0].label_outer()
        img = librosa.display.specshow(mfcc, x_axis='time', ax=ax[1])
        fig.colorbar(img, ax=[ax[1]])
        ax[1].set(title='MFCC')
        plt.show()
        if(index>20):
          break
        index+=1


transcripts_files: 97
audio_files: 108


  0%|          | 0/7 [00:04<?, ?it/s]


AttributeError: ignored

* https://pytorch.org/audio/stable/generated/torchaudio.models.emformer_rnnt_model.html?highlight=rnnt#torchaudio.models.emformer_rnnt_model

In [ ]:
net = emformer_rnnt_model(
        input_dim=96,
        encoding_dim=1024,
        num_symbols=34,
        segment_length=16,
        right_context_length=4,
        time_reduction_input_dim=128,
        time_reduction_stride=4,
        transformer_num_heads=8,
        transformer_ffn_dim=2048,
        transformer_num_layers=20,
        transformer_dropout=0.1,
        transformer_activation="gelu",
        transformer_left_context_length=30,
        transformer_max_memory_size=0,
        transformer_weight_init_scale_strategy="depthwise",
        transformer_tanh_on_mem=True,
        symbol_embedding_dim=512,
        num_lstm_layers=3,
        lstm_layer_norm=True,
        lstm_layer_norm_epsilon=1e-3,
        lstm_dropout=0.3,
    )

* https://pytorch.org/audio/stable/generated/torchaudio.models.RNNTBeamSearch.html?highlight=beamsearch#torchaudio.models.RNNTBeamSearch

In [ ]:
def predict(net,dataset):
  net.eval()
  state = None
  infer = torchaudio.models.RNNTBeamSearch(net,blank=0)
  with torch.no_grad():
    for (features,feat_lens,labels,label_lens) in dataset:
      feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)

      features = nn.utils.rnn.pad_sequence(features,batch_first=True)
      labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
      
      output = infer(features.float(),feat_lens,10)
      
      tmp_output = output[0]
      
      print("-"*10)
      print("\toriginal:",labels)
      #with_visible_tag
      ori_string = dataset.token2String(labels.numpy()[0].tolist())
      pred_string = dataset.token2String(output[0][0])
      print("\tori string: ",ori_string)
      print("\toutput result:",output[0][0])
      print("\topt string: ",pred_string)
      ori_string = dataset.token2String(labels.numpy()[0].tolist(),with_visible_tag=False)
      pred_string = dataset.token2String(output[0][0],with_visible_tag=False)
      print(f"\topt' string: '{pred_string}'")
      print("\twer:",wer(pred_string,ori_string))
      print("\tcer:",cer(pred_string,ori_string))
  

In [ ]:
initial_learning_rate = 0.0004
optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=100,eta_min=0.001)

In [ ]:
if(__name__=="__main__"):
    #initial_learning_rate = 0.0001
    max_num_epoch = 5
    #optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
    criterion = audioT.RNNTLoss(blank=0,fused_log_softmax=False,reduction='sum')#,clamp=1.0)#'sum')
    traindataset = DataLoader(3)
    testdataset = DataLoader(1,is_test_set=True)
    #torch.save(net.parameters(),"rnnt.pth")
    #predict(net,testdataset)
    net.train()
    for epoch in range(max_num_epoch):
        index = 0
        traindataset.shuffleDataset()
        losses = []
        for (features,feat_lens,labels,label_lens) in tqdm((traindataset)):
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)

            features = nn.utils.rnn.pad_sequence(features,batch_first=True)#.unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)

            optimizer.zero_grad()

            joint_outputs,src_length,tgt_length,*_ = net(features.float(),feat_lens,labels,label_lens)
            
            outputs = nn.functional.log_softmax(joint_outputs,dim=-1)

            loss = criterion(outputs,(labels[...,1:]).int(),(src_length-1).int() ,(label_lens-1).int())

            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            if(index%100==0):
                print(loss.item())
            index+=1
        print(f"ep={epoch}] loss= ",np.mean(losses))
        scheduler.step()
    torch.save(net.state_dict(),"rnnt.pth")
    predict(net,testdataset)

transcripts_files: 97
audio_files: 108
transcripts_files: 87
audio_files: 2620


  3%|▎         | 1/36 [00:04<02:30,  4.30s/it]

169.48074340820312


100%|██████████| 36/36 [04:00<00:00,  6.68s/it]


ep=0] loss=  254.54145050048828


...

  3%|▎         | 1/36 [00:03<02:07,  3.65s/it]

106.3607177734375


100%|██████████| 36/36 [03:46<00:00,  6.30s/it]


ep=4] loss=  155.9362850189209
----------
	original: tensor([[ 0, 17, 16, 29, 21,  3, 22, 23, 20,  6,  3, 27, 29, 15, 17, 20, 16, 11,
         16,  9, 21, 29, 25, 10,  7, 16, 29, 22, 10,  7, 29, 21, 17,  6,  3, 14,
         11, 22, 27, 29, 15,  7, 22, 29, 11, 16, 29, 22, 10,  7, 29,  5, 10,  3,
         18,  7, 14, 29, 22, 17, 29, 20,  7,  5, 11, 22,  7, 29, 22, 10,  7, 29,
         14, 11, 22, 22, 14,  7, 29, 17,  8,  8, 11,  5,  7, 29, 10, 11, 21, 29,
         18, 14,  3,  5,  7, 29, 25,  3, 21, 29,  3, 29,  5, 23, 21, 10, 11, 17,
         16,  7,  6, 29, 13, 16,  7,  7, 14, 11, 16,  9, 29,  6,  7, 21, 13, 29,
          3, 22, 29, 22, 10,  7, 29, 20, 11,  9, 10, 22, 29, 17,  8, 29, 22, 10,
          7, 29,  3, 14, 22,  3, 20, 29,  8, 20, 17, 15, 29, 25, 10, 11,  5, 10,
         29, 10,  7, 29, 14,  7,  6, 29, 10, 11, 21, 29, 25, 11, 16,  9, 29, 17,
          8, 29,  4, 17, 27, 21, 29, 22, 10, 20, 17, 23,  9, 10, 29, 22, 10,  7,
         29, 20,  7, 21, 18, 17, 16, 21,  7, 21,  0]])
	

KeyboardInterrupt: ignored